# Data-Driven Strategies for a Successful Movie Studio Launch

# 🎬 Business Understanding

## 📘 Business Context
Our company is entering the competitive movie production industry by establishing a new movie studio. With major corporations increasingly investing in original video content, we need to strategically position ourselves to maximize our chances of box office success.

## 👤 Primary Stakeholder
**Head of New Movie Studio** — The key decision-maker who will use our analysis to guide the studio's content creation strategy and investment decisions.

## ❓ Key Business Questions

### 1. **What genres are most profitable?**
- Which movie genres consistently generate the highest box office returns?
- Are there emerging genres showing strong growth potential?

### 2. **What budget ranges optimize ROI?**
- What is the relationship between production budget and box office performance?
- What budget sweet spot maximizes return on investment?

### 3. **When should we release our films?**
- Which months/seasons show the strongest box office performance?
- How can we avoid oversaturated release periods?

## 🎯 Business Objectives
- **Primary Goal:** Identify the most profitable film characteristics to guide our initial movie productions.
- **Success Metrics:** Maximize box office revenue and return on investment (ROI).
- **Risk Mitigation:** Minimize financial losses by avoiding poorly-performing film categories.

## 📊 Data Requirements & Analysis Considerations

### 📂 Dataset Specifications Needed:
1. **What movie data do we have access to?** (e.g., box office numbers, budgets, genres, release dates, ratings)
2. **What time period does our data cover?** (Recent years or historical data?)
3. **Is it domestic, international, or worldwide box office gross?**

### 📎 Business Context Clarifications:
4. **What size studio are we targeting?** (Independent/small budget vs. major studio competitor?)
5. **Are there any genre restrictions?** (Family-friendly only, or open to all ratings?)
6. **Geographic focus?** (Domestic US market, international, or global?)

### 📌 Analysis Scope Considerations:
7. **Should we consider streaming/digital revenue or just theatrical box office?**
8. **Do we want to analyze competitor studios' strategies?**
9. **Should seasonal trends consider the impact of COVID-19 in recent years?**

## 📈 Success Metrics Framework

### Primary Metrics:
- **Gross Revenue:** Total box office performance
- **Return on Investment (ROI):** \((\text{Revenue} - \text{Budget}) / \text{Budget} \times 100\)
- **Profit Margins:** Net profit as percentage of revenue
- **Budget Efficiency:** Revenue per dollar invested

### Secondary Metrics:
- **Market Share:** Performance relative to industry
- **Risk Assessment:** Variance in returns by category
- **Audience Reach:** Ticket sales and demographic penetration


# 🧠 Data Understanding

To analyze what makes a movie successful, we are working with multiple datasets from trusted sources like **Box Office Mojo**, **IMDb**, **Rotten Tomatoes**, **The Movie Database (TMDb)**, and **The Numbers**. Below is an overview of each dataset and the structure of the data they contain.

---

## 🎬 `bom.movie_gross.csv.gz` (Box Office Mojo)

This dataset contains revenue data for various films.

**Columns:**
- `title`: Movie title
- `studio`: Producing studio
- `domestic_gross`: Domestic gross earnings (USD)
- `foreign_gross`: International gross earnings
- `year`: Year of release

---

## 🎥 IMDb Database (`im.db`)

This is a zipped SQLite database containing multiple related tables from IMDb. Below are the most relevant tables and their fields:

### 🔹 `directors`
- `movie_id`: Unique movie ID
- `person_id`: Director’s ID

### 🔹 `known_for`
- `person_id`: Unique person ID
- `movie_id`: Associated movie ID

### 🔹 `movie_akas`
- `movie_id`: Unique movie ID
- `ordering`: Title order
- `title`: Alternate movie title
- `region`: Country code
- `language`: Language code
- `types`: Type of title (e.g. working title, festival name)
- `attributes`: Special title attributes
- `is_original_title`: Whether it is the original title (1 = yes)

### 🔹 `movie_basics`
- `movie_id`: Unique identifier
- `primary_title`: Official movie title
- `original_title`: Title in original language
- `start_year`: Release year
- `runtime_minutes`: Duration of movie
- `genres`: Genres (comma-separated if multiple)

### 🔹 `movie_ratings`
- `movie_id`: Unique movie ID
- `averagerating`: IMDb average rating
- `numvotes`: Total number of votes

### 🔹 `persons`
- `person_id`: Unique person ID
- `primary_name`: Person's name
- `birth_year`: Year of birth
- `death_year`: Year of death (if applicable)
- `primary_profession`: Role in industry (e.g., actor, director)

### 🔹 `principals`
- `movie_id`: Unique movie ID
- `ordering`: Credit order
- `person_id`: Unique person ID
- `category`: Crew or cast category
- `job`: Specific job title
- `characters`: Character(s) played

### 🔹 `writers`
- `movie_id`: Unique movie ID
- `person_id`: Writer’s ID

---

## 🍅 `rt.movie_info.tsv.gz` (Rotten Tomatoes - Movie Info)

General information about movies.

**Columns:**
- `id`: Movie ID (RT)
- `synopsis`: Movie summary
- `rating`: MPAA Rating
- `genre`: Movie genres
- `director`: Director name(s)
- `writer`: Writer name(s)
- `theater_date`: Theatrical release date
- `dvd_date`: DVD release date
- `currency`: Currency symbol
- `box_office`: Total box office revenue
- `runtime`: Duration in minutes
- `studio`: Producing studio

---

## 📝 `rt.reviews.tsv.gz` (Rotten Tomatoes - Reviews)

Contains individual critic reviews and associated metadata.

**Columns:**
- `id`: Movie ID (RT)
- `review`: Review text
- `rating`: Rating given
- `fresh`: Review sentiment (fresh = positive, rotten = negative)
- `critic`: Name of the critic
- `top_critic`: Is the critic a "Top Critic"? (1 = yes, 0 = no)
- `publisher`: Publication name
- `date`: Review date

---

## 🎞️ `tmdb.movies.csv.gz` (The Movie Database)

Contains movie metadata and popularity data from TMDb.

**Columns:**
- `genre_ids`: List of genre IDs
- `id`: Movie ID (TMDb)
- `original_language`: Language code
- `original_title`: Title in original language
- `popularity`: Popularity score
- `release_date`: Release date
- `title`: English movie title
- `vote_average`: Average user rating
- `vote_count`: Number of votes

---

## 💰 `tn.movie_budgets.csv.gz` (The Numbers)

Contains production budget and revenue details.

**Columns:**
- `id`: Movie ID (The Numbers)
- `release_date`: Theatrical release date
- `movie`: Movie title
- `production_budget`: Budget in USD
- `domestic_gross`: Domestic earnings in USD
- `worldwide_gross`: Total earnings in USD

---

## 🔍 Next Steps

We will explore, clean, and merge relevant data from these datasets to:
- Compare budget and revenue
- Analyze genre-based performance
- Evaluate seasonal release patterns
- Identify high-performing studios, ratings, and durations


In [1]:
#importing relevant libs
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import gauss
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
import sqlite3

## Loading The Provided Data

In [2]:
#going through SQl
def load_tables(connection):
    table_names = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", connection
    )["name"].tolist()
    sql_tables = {
        table_name: pd.read_sql(f"SELECT * FROM {table_name}", connection)
        for table_name in table_names
    }
    return sql_tables

path = '../Data/im.db'
conn = sqlite3.connect(path)

imdb_tables = load_tables(conn)

print(imdb_tables.keys())

#-------CSV and TSV Data ------#

bom_movie = pd.read_csv("../Data/bom.movie_gross.csv")
rt_info = pd.read_csv("../Data/rt.movie_info.tsv", sep='\t', encoding='latin1')
rt_reviews = pd.read_csv("../Data/rt.movie_info.tsv", sep='\t', encoding='latin1')
tmdb_df = pd.read_csv("../Data/tmdb.movies.csv")
tn_budget = pd.read_csv("../Data/tn.movie_budgets.csv")

dict_keys(['movie_basics', 'directors', 'known_for', 'movie_akas', 'movie_ratings', 'persons', 'principals', 'writers'])


## Cleaning and Understanding `tn_budget`

In [3]:
budget = tn_budget.copy(deep=True)
print(budget.shape)
print(budget.info())
budget.head()
budget.tail()
print(budget.columns)
budget.duplicated().sum()
budget.drop(columns=['id'])

(5782, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB
None
Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')


,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [4]:
budget = budget.drop(columns=['id'])

## Renaming Columns and Cleaning

In [5]:
budget = budget.rename( columns={
    "movie": "movie_title"
})

budget['release_date'] = budget['release_date'].astype(str).str.strip()
budget['release_date'] = pd.to_datetime(budget['release_date'], format='%b %d, %Y')


In [6]:
finance_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in finance_cols:
    budget[col] = (budget[col].replace('[\$,]','', regex=True).astype(float))

In [7]:
def clean_title(df, title_col='movie_title'):
    df[title_col] = df[title_col].str.lower()                     
    df[title_col] = df[title_col].str.strip()                      
    df[title_col] = df[title_col].str.replace(r'[^a-z0-9 ]', '', regex=True)  
    return df

In [8]:
clean_title(budget, 'movie_title')

,release_date,movie_title,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,avatar,425000000.0,760507625.0,2.776345e+09
1,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09
2,2019-06-07,dark phoenix,350000000.0,42762350.0,1.497624e+08
3,2015-05-01,avengers age of ultron,330600000.0,459005868.0,1.403014e+09
4,2017-12-15,star wars ep viii the last jedi,317000000.0,620181382.0,1.316722e+09
...,...,...,...,...,...
5777,2018-12-31,red 11,7000.0,0.0,0.000000e+00
5778,1999-04-02,following,6000.0,48482.0,2.404950e+05
5779,2005-07-13,return to the land of wonders,5000.0,1338.0,1.338000e+03
5780,2015-09-29,a plague so pleasant,1400.0,0.0,0.000000e+00


In [9]:
budget

,release_date,movie_title,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,avatar,425000000.0,760507625.0,2.776345e+09
1,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09
2,2019-06-07,dark phoenix,350000000.0,42762350.0,1.497624e+08
3,2015-05-01,avengers age of ultron,330600000.0,459005868.0,1.403014e+09
4,2017-12-15,star wars ep viii the last jedi,317000000.0,620181382.0,1.316722e+09
...,...,...,...,...,...
5777,2018-12-31,red 11,7000.0,0.0,0.000000e+00
5778,1999-04-02,following,6000.0,48482.0,2.404950e+05
5779,2005-07-13,return to the land of wonders,5000.0,1338.0,1.338000e+03
5780,2015-09-29,a plague so pleasant,1400.0,0.0,0.000000e+00


## Cleaning and Understanding `tmdb_df`

In [10]:
tmdb = tmdb_df.copy(deep=True)
print(tmdb.shape)
print(tmdb.info())
tmdb.head()
tmdb.tail()
print(tmdb.columns)
tmdb.duplicated().sum()



(26517, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB
None
Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')


np.int64(0)

In [11]:
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


## Understanding, Dropping and Cleaning

In [12]:
#Dropping 

tmdb = tmdb.drop(columns=['Unnamed: 0', 'genre_ids', 'id', 'title'])

In [13]:
tmdb.head()

,original_language,original_title,popularity,release_date,vote_average,vote_count
0,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,7.7,10788
1,en,How to Train Your Dragon,28.734,2010-03-26,7.7,7610
2,en,Iron Man 2,28.515,2010-05-07,6.8,12368
3,en,Toy Story,28.005,1995-11-22,7.9,10174
4,en,Inception,27.920,2010-07-16,8.3,22186


In [14]:
tmdb = tmdb.rename(columns={
    "original_title": "movie_title"
})

clean_title(tmdb, 'movie_title')

,original_language,movie_title,popularity,release_date,vote_average,vote_count
0,en,harry potter and the deathly hallows part 1,33.533,2010-11-19,7.7,10788
1,en,how to train your dragon,28.734,2010-03-26,7.7,7610
2,en,iron man 2,28.515,2010-05-07,6.8,12368
3,en,toy story,28.005,1995-11-22,7.9,10174
4,en,inception,27.920,2010-07-16,8.3,22186
...,...,...,...,...,...,...
26512,en,laboratory conditions,0.600,2018-10-13,0.0,1
26513,en,exhibit84xxx,0.600,2018-05-01,0.0,1
26514,en,the last one,0.600,2018-10-01,0.0,1
26515,en,trailer made,0.600,2018-06-22,0.0,1


In [15]:
tmdb['release_date'] = tmdb['release_date'].astype(str).str.strip()
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'], format='ISO8601')

In [16]:
tmdb.head()

,original_language,movie_title,popularity,release_date,vote_average,vote_count
0,en,harry potter and the deathly hallows part 1,33.533,2010-11-19,7.7,10788
1,en,how to train your dragon,28.734,2010-03-26,7.7,7610
2,en,iron man 2,28.515,2010-05-07,6.8,12368
3,en,toy story,28.005,1995-11-22,7.9,10174
4,en,inception,27.920,2010-07-16,8.3,22186


In [ ]:
language_map = {
    'en': 'English',
    'nl': 'Dutch',
    'es': 'Spanish',
    'ja': 'Japanese',
    'sv': 'Swedish',
    'de': 'German',
    'fr': 'French',
    'cn': 'Chinese (unspecified)',  
    'it': 'Italian',
    'ru': 'Russian',
    'zh': 'Chinese',
    'hi': 'Hindi',
    'no': 'Norwegian',
    'ko': 'Korean',
    'da': 'Danish',
    'fi': 'Finnish',
    'pl': 'Polish',
    'te': 'Telugu',
    'hu': 'Hungarian',
    'tr': 'Turkish',
    'pt': 'Portuguese',
    'he': 'Hebrew',
    'fa': 'Persian (Farsi)',
    'th': 'Thai',
    'cs': 'Czech',
    'et': 'Estonian',
    'tl': 'Tagalog',
    'lt': 'Lithuanian',
    'xx': 'No Language / Unknown',
    'bs': 'Bosnian',
    'ar': 'Arabic',
    'is': 'Icelandic',
    'el': 'Greek',
    'mr': 'Marathi',
    'hr': 'Croatian',
    'ro': 'Romanian',
    'sr': 'Serbian',
    'uk': 'Ukrainian',
    'nb': 'Norwegian Bokmål',
    'hz': 'Herero',
    'ca': 'Catalan',
    'bg': 'Bulgarian',
    'sl': 'Slovenian',
    'lv': 'Latvian',
    'si': 'Sinhala',
    'ab': 'Abkhazian',
    'ta': 'Tamil',
    'bo': 'Tibetan',
    'id': 'Indonesian',
    'sq': 'Albanian',
    'bn': 'Bengali',
    'gu': 'Gujarati',
    'lo': 'Lao',
    'ne': 'Nepali',
    'kk': 'Kazakh',
    'hy': 'Armenian',
    'ps': 'Pashto',
    'kn': 'Kannada',
    'vi': 'Vietnamese',
    'ku': 'Kurdish',
    'ka': 'Georgian',
    'ml': 'Malayalam',
    'ur': 'Urdu',
    'mi': 'Maori',
    'eu': 'Basque',
    'sn': 'Shona',
    'cy': 'Welsh',
    'ha': 'Hausa',
    'ky': 'Kyrgyz',
    'yi': 'Yiddish',
    'pa': 'Punjabi',
    'xh': 'Xhosa',
    'cr': 'Cree',
    'sw': 'Swahili',
    'af': 'Afrikaans',
    'dz': 'Dzongkha'
}
tmdb['original_language'] = tmdb['original_language'].map(language_map)

,original_language,movie_title,popularity,release_date,vote_average,vote_count
0,English,harry potter and the deathly hallows part 1,33.533,2010-11-19,7.7,10788
1,English,how to train your dragon,28.734,2010-03-26,7.7,7610
2,English,iron man 2,28.515,2010-05-07,6.8,12368
3,English,toy story,28.005,1995-11-22,7.9,10174
4,English,inception,27.920,2010-07-16,8.3,22186
...,...,...,...,...,...,...
26512,English,laboratory conditions,0.600,2018-10-13,0.0,1
26513,English,exhibit84xxx,0.600,2018-05-01,0.0,1
26514,English,the last one,0.600,2018-10-01,0.0,1
26515,English,trailer made,0.600,2018-06-22,0.0,1
